**Importing required libraries**

# Bayesian Models

In [21]:
import pymc3 as pm

with pm.Model() as model_besag_1:
    
    beta_unique_tower_count = pm.Normal('beta_unique_tower_count', mu=0, sigma=10)
    beta_pregnancies_log = pm.Normal('beta_pregnancies_log', mu=0, sigma=10)
    beta_social_entropy = pm.Normal('beta_social_entropy', mu=0, sigma=10)
    tau = pm.Gamma('tau', alpha=0.5, beta=0.5)  # precision parameter for the Besag model
    

    eta = pm.Normal('eta', mu=0, sigma=1, shape=len(gdf)) 
    
    mu = (
        beta_unique_tower_count * gdf['unique_tower_count'] +
        beta_pregnancies_log * gdf['pregnancies_log'] +
        beta_social_entropy * gdf['social_entropy'] +
        eta +
        neighbour_all  #
    )
    #likelihood
    pc_1_na = pm.Poisson('pc_1', mu=np.exp(mu), observed=gdf['pc_1'])

    # Sample posterior
    trace = pm.sample(1000, tune=1000)

print('')

AttributeError: partially initialized module 'theano' has no attribute 'compile' (most likely due to a circular import)

In [ ]:
#validating model againset DSD level poverty headcount

dsd_poverty = pd.read_excel("/Users/chanukaalgama/Desktop/DAP PROJECTS/RS + CDR Poverty Mapping/poverty-mapping-liao-220902/dsd-poverty.xlsx")

census = pd.read_csv("/Users/chanukaalgama/Desktop/DAP PROJECTS/RS + CDR Poverty Mapping/poverty-mapping-liao-220902/data/census_features.csv")
census = census.rename(columns={"gnd_id": "gnd_id", "age_total": "pop"})

data_full_with_id = merged[["gnd_id", "pc_1"]]

fit_besag_full = pd.DataFrame({"fit": model_besag_1.summary["mean"]})
fit_besag_full = pd.concat([fit_besag_full, data_full_with_id], axis=1)

census_merged = pd.merge(fit_besag_full, census, on="gnd_id")


gnd2dsd_df = data_shapefile.drop(columns=["geometry"]).rename(columns={"code_7": "gnd_id"})
gnd2dsd_df = pd.merge(gnd2dsd_df, census_merged, on="gnd_id")

gnd2dsd_gr = gnd2dsd_df.groupby("dsd_name").apply(lambda x: np.average(x["fit"], weights=x["pop"])).reset_index(name="fit_wted")

gnd2dsd_gr = pd.merge(gnd2dsd_gr, dsd_poverty, on="dsd_name", how="outer")


final_df = pd.read_csv("gnd2dsd-gr-cln-final.csv")
final_df["fit_wted_std"] = (final_df["fit_wted"] - final_df["fit_wted"].mean()) / final_df["fit_wted"].std()
final_df["hci_std"] = (final_df["headcount_index"].mean() - final_df["headcount_index"]) / final_df["headcount_index"].std()
final_df["res"] = final_df["fit_wted_std"] - final_df["hci_std"]

RRes_dsd = np.sum(final_df["res"].dropna() ** 2)
RRtot_dsd = np.sum((final_df["hci_std"] - final_df["fit_wted_std"].mean()) ** 2)

pseudo_r2_dsd = 1 - RRes_dsd / RRtot_dsd

# Spearman correlation
spearman_corr = spearmanr(final_df["hci_std"], final_df["fit_wted_std"])

 
lm_hci_std = LinearRegression().fit(final_df[["fit_wted_std"]], final_df["hci_std"])
lm_headcount_index = LinearRegression().fit(final_df[["fit_wted"]], final_df["headcount_index"])


print("Pseudo R-squared (DSD):", pseudo_r2_dsd)
print("Spearman correlation:", spearman_corr.correlation)
print("Spearman correlation p-value:", spearman_corr.pvalue)
print("Linear regression (hci_std ~ fit_wted_std) coefficients:", lm_hci_std.coef_)
print("Linear regression (headcount_index ~ fit_wted) coefficients:", lm_headcount_index.coef_)
